In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import re

In [2]:
df = pd.read_csv("./Recipe/IndianFoodDatasetCSV.csv")
def clean_ingredients(text):
    if pd.isna(text):
        return ""
    # Remove quantities like "2 cups", "3 tbsp", "1 teaspoon"
    text = re.sub(r"\b\d+[\w\s/]*\b", "", text)
    # Remove commas, parentheses, etc.
    text = re.sub(r"[\(\)\[\]]", "", text)
    # Remove measurement words
    text = re.sub(r"\b(cup|cups|tablespoon|tbsp|teaspoon|tsp|grams|g|kg|ml|ltr|pinch|piece|pieces|slice|slices|cloves?|nos?|small|medium|large)\b", "", text, flags=re.IGNORECASE)
    # Remove extra spaces and punctuation
    text = re.sub(r"[^a-zA-Z,\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    # Split by commas and remove duplicates
    ingredients = list(dict.fromkeys([i.strip() for i in text.split(",") if i.strip()]))
    return ", ".join(ingredients)

# Apply cleaning
df["CleanedIngredients"] = df["TranslatedIngredients"].apply(clean_ingredients)

df[["TranslatedIngredients", "CleanedIngredients"]].head()

def split_steps(text):
    if pd.isna(text):
        return []
    # Split by sentence or numbered steps
    steps = re.split(r'(?:(?<=\.)|(?<=\d\)))\s+', text)
    steps = [step.strip() for step in steps if len(step.strip()) > 2]
    return steps

df["RecipeSteps"] = df["TranslatedInstructions"].apply(split_steps)

# Show example
df[["TranslatedInstructions", "RecipeSteps"]].head()

# Save cleaned data
df.to_csv("cleaned_indian_food.csv", index=False)

print("✅ Cleaned dataset saved as 'cleaned_indian_food.csv'")


✅ Cleaned dataset saved as 'cleaned_indian_food.csv'


In [3]:
df = pd.read_csv("cleaned_indian_food.csv")

# Step 3: Basic preprocessing
df.dropna(subset=["CleanedIngredients"], inplace=True)
df.reset_index(drop=True, inplace=True)

# Step 4: TF-IDF Vectorization on cleaned ingredients
vectorizer = TfidfVectorizer(stop_words="english")
ingredient_vectors = vectorizer.fit_transform(df["CleanedIngredients"])

In [4]:
def recommend_recipe(user_ingredients, top_n=5):
    """
    Recommend similar recipes based on ingredient similarity.
    user_ingredients: list or comma-separated string of ingredients
    top_n: number of results to return
    """
    if isinstance(user_ingredients, list):
        user_input = ", ".join(user_ingredients)
    else:
        user_input = user_ingredients

    user_vector = vectorizer.transform([user_input])
    similarity = cosine_similarity(user_vector, ingredient_vectors).flatten()

    indices = np.argsort(similarity)[::-1][:top_n]
    recommendations = df.iloc[indices][["TranslatedRecipeName", "CleanedIngredients", "RecipeSteps", "Cuisine", "Course", "Diet"]]
    
    return recommendations

# Step 6: Function to neatly display recipe steps
def show_recipe_steps(recipe_name):
    """
    Display the steps of a recipe given its name.
    """
    recipe = df[df["TranslatedRecipeName"].str.lower() == recipe_name.lower()]
    if recipe.empty:
        print("❌ Recipe not found.")
        return
    steps = eval(recipe.iloc[0]["RecipeSteps"]) if isinstance(recipe.iloc[0]["RecipeSteps"], str) else recipe.iloc[0]["RecipeSteps"]
    print(f"\n👨‍🍳 Steps for {recipe_name}:\n")
    for i, step in enumerate(steps, start=1):
        print(f"Step {i}: {step}")

In [5]:
user_input = "onion, tomato, garlic, ginger"
print(f"\n🔍 Top recipe recommendations for: {user_input}\n")

recommendations = recommend_recipe(user_input, top_n=5)
display(recommendations)

# Step 8: Show recipe steps (example)
show_recipe_steps(recommendations.iloc[0]["TranslatedRecipeName"])


🔍 Top recipe recommendations for: onion, tomato, garlic, ginger



,TranslatedRecipeName,CleanedIngredients,RecipeSteps,Cuisine,Course,Diet
750,Grilled Chicken With Vegetables Recipe,"Salt to taste, crushed, Ginger Garlic Paste as...",['To begin making the Grilled Chicken With Veg...,Indian,Dinner,High Protein Non Vegetarian
6219,Nagaland Style Kali Dal (Recipe In Hindi),"split, baking soda, salt as per taste, Ginger ...","['To make Nagaland style black lentils, first ...",Nagaland,Lunch,Vegetarian
3127,Mustard Mayo Baby Potato Salad Recipe,"finely chopped, Capsicum finely chopped, Tomat...",['To begin making the Mustard Mayo Baby Potato...,Continental,Appetizer,Vegetarian
865,Thai Yellow Curry Recipe,"I used potato, Thai basil leaves few s optiona...","['To prepare Thai Yellow Curry recipe, begin w...",Thai,Lunch,Vegetarian
3176,Suran Jo Kheemo Recipe-Sindhi Style Grated Yam...,"SuranSenaiRatalu grated or finely chopped, Jee...","['To begin making the Suran Jo Kheemo recipe, ...",Sindhi,Lunch,Vegetarian



👨‍🍳 Steps for Grilled Chicken With Vegetables Recipe:

Step 1: To begin making the Grilled Chicken With Vegetables recipe, firstly we will saute the vegetables.Heat olive oil in a skillet/stir fry pan.
Step 2: Add onion, pounded garlic and saute it for a minute or two.Next, add all the vegetables including red pepper, green pepper, broccoli, cherry tomatoes, onions and baby corns.Sprinkle some salt over it and cover the pan with a lid.
Step 3: Let it cook for 3 to 4 minutes more.Add mixed herbs and crushed peppercorn.
Step 4: Mix everything well and keep it aside.The next step is to prepare the grilled chicken.Take chicken breast and pat it slightly so it becomes flat and can be cooked evenly.Sprinkle salt, crushed black pepper, dry basil leaves and red chilli flakes and rub it properly on both side.
Step 5: Apply ginger garlic paste on the both the sides as well.Drizzle some olive oil on it and rub it with mixture.
Step 6: Make sure all these steps which are mentioned above, we need 